In [1]:
# Instalando bibliotecas necessárias no Jupter
%pip install pandas sqlalchemy psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached sqlalchemy-2.0.44-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached psycopg2_binary-2.9.11-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached numpy-2.3.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached greenlet-3.2.4-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached sqlalchemy-2.0.44-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached psycopg2_binary-2.9.11-cp312-cp312-win_amd64.whl (2.7 MB)
Using cached greenlet-3.2.4-cp312-cp312-win_amd64.whl (299 kB)
Using cached numpy-2.3.5-cp312-cp312-win_amd64.whl (12.8 MB)
Using cached pytz


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Celso\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# CÉLULA 2.0: Extração de Dados (Com Download Automático)

import pandas as pd
import os
import gdown  # Biblioteca para baixar arquivos do Google Drive

# --- CONFIGURAÇÃO ---
# Caminho da pasta onde os arquivos estão (ou serão baixados)
caminho_base = r'C:\Users\Celso\Downloads'

# Mapeamento dos Arquivos Locais (Nome que será salvo/lido)
arquivos = {
    2021: 'atendimentoturismo2021.csv',
    2022: 'atendimentoturismo2022.csv',
    2023: 'atendimentoturismo2023.csv'
}

# Mapeamento dos IDs do Google Drive (Extraídos dos seus links)
drive_ids = {
    2021: '1sOiEizQLfYH_R2yfuL_6vwm-B4C_rItE',
    2022: '1-0Yeze_VG4dVAgrFluK-a-3kEvryHi40',
    2023: '1EABOwDWcQWnBeD8EfbrBqMQKHxJq6DOb'
}

# --- EXTRAÇÃO ---
dfs = []

# Garante que a pasta existe
if not os.path.exists(caminho_base):
    try:
        os.makedirs(caminho_base)
        print(f"📁 Pasta criada: {caminho_base}")
    except:
        print(f"⚠️ Não foi possível criar a pasta. Usando diretório atual.")
        caminho_base = '.'

for ano, nome_arquivo in arquivos.items():
    caminho_completo = os.path.join(caminho_base, nome_arquivo)
    
    # 1. VERIFICAÇÃO E DOWNLOAD (FALLBACK)
    if not os.path.exists(caminho_completo):
        print(f"📥 Arquivo de {ano} não encontrado localmente. Baixando do Drive...")
        try:
            file_id = drive_ids.get(ano)
            if file_id:
                url = f'https://drive.google.com/uc?id={file_id}'
                gdown.download(url, caminho_completo, quiet=False)
                print(f"   -> Download concluído: {nome_arquivo}")
            else:
                print(f"   -> ID do Drive não encontrado para {ano}.")
        except Exception as e_download:
            print(f"   ❌ Falha ao baixar arquivo: {e_download}")

    # 2. LEITURA DO ARQUIVO
    if os.path.exists(caminho_completo):
        try:
            # Tenta ler com separador ';' e encoding utf-8
            df_temp = pd.read_csv(caminho_completo, sep=';', encoding='utf-8', on_bad_lines='skip')
            
            # NOTA: Não criamos 'ano_referencia' pois o arquivo já possui a coluna 'ano'
            
            dfs.append(df_temp)
            print(f"✅ {ano}: Carregado com sucesso ({len(df_temp)} registros)")
            
        except Exception as e:
            # Se der erro de encoding, tenta 'latin1'
            try:
                df_temp = pd.read_csv(caminho_completo, sep=';', encoding='latin1', on_bad_lines='skip')
                
                # NOTA: Não criamos 'ano_referencia' pois o arquivo já possui a coluna 'ano'
                
                dfs.append(df_temp)
                print(f"✅ {ano} (via latin1): Carregado com sucesso ({len(df_temp)} registros)")
            except Exception as e2:
                print(f"❌ Erro crítico ao ler {nome_arquivo}: {e2}")
    else:
        print(f"⚠️ Arquivo indisponível para {ano} (Falha no Download ou Local).")

# Unifica todos os anos em um único DataFrame
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"\n📊 Base consolidada criada com sucesso! Total de linhas: {len(df_final)}")
else:
    print("\n❌ Nenhum dado foi carregado.")

✅ 2021: Carregado com sucesso (15374 registros)
✅ 2022: Carregado com sucesso (37510 registros)
✅ 2023: Carregado com sucesso (22239 registros)

📊 Base consolidada criada com sucesso! Total de linhas: 75123


In [31]:
# CÉLULA 3.0: Transformação (Corrigida e Limpa)

if 'df_final' in locals() and not df_final.empty:
    print("🔄 Iniciando transformações...")

    # 1. Padronização dos Nomes das Colunas
    # Remove acentos, espaços e deixa tudo minúsculo
    df_final.columns = (df_final.columns
                        .str.strip()
                        .str.lower()
                        .str.replace(' ', '_')
                        .str.replace('ç', 'c')
                        .str.replace('ã', 'a')
                        .str.replace('õ', 'o')
                        .str.replace('á', 'a')
                        .str.replace('é', 'e')
                        .str.replace('í', 'i')
                        .str.replace('ú', 'u')
                       )
    
    print("   -> Nomes das colunas padronizados.")

    # 2. Correção Crítica da Coluna ANO (Remove decimais como 2022.1 ou 2022.0)
    if 'ano' in df_final.columns:
        # Primeiro, garante que é número (transforma erros em NaN)
        df_final['ano'] = pd.to_numeric(df_final['ano'], errors='coerce')
        
        # Preenche vazios com 0 e converte para INTEIRO (remove o ponto decimal)
        df_final['ano'] = df_final['ano'].fillna(0).astype(int)
        
        # Opcional: Se quiser garantir que o '0' vire nulo novamente depois, 
        # mas para remover o decimal, o int é obrigatório.
        print("   -> Coluna 'ano' corrigida para formato Inteiro (sem decimais).")

    # 3. Tratamento de Datas (Apenas converte, não cria colunas novas)
    colunas_data = ['data', 'data_atendimento', 'dt_registro']
    for col in colunas_data:
        if col in df_final.columns:
            df_final[col] = pd.to_datetime(df_final[col], errors='coerce', dayfirst=True)

    # 4. Limpeza de Textos
    cols_texto = df_final.select_dtypes(include=['object']).columns
    for col in cols_texto:
        df_final[col] = df_final[col].str.strip().str.upper()
        df_final[col] = df_final[col].fillna('NAO INFORMADO')
        df_final[col] = df_final[col].replace(['NAN', 'NULL', '', 'nan'], 'NAO INFORMADO')

    print("✅ Transformações concluídas.")
    
    # Validação visual imediata
    if 'ano' in df_final.columns:
        print("\nExemplo da coluna ANO (deve estar como número inteiro):")
        print(df_final['ano'].unique())

else:
    print("❌ Erro: O DataFrame 'df_final' não existe.")

🔄 Iniciando transformações...
   -> Nomes das colunas padronizados.
   -> Coluna 'ano' corrigida para formato Inteiro (sem decimais).
✅ Transformações concluídas.

Exemplo da coluna ANO (deve estar como número inteiro):
[2021 2022 2023]


In [32]:
# CÉLULA 3.1: Prévia dos Dados (Com Correção de Erros)

if 'df_final' in locals() and not df_final.empty:
    # --- CORREÇÃO AUTOMÁTICA ---
    # Esta linha remove qualquer coluna duplicada que tenha sobrado na memória
    df_final = df_final.loc[:, ~df_final.columns.duplicated()]
    
    print("-" * 50)
    print(f"📊 Total de Linhas: {df_final.shape[0]}")
    print(f"📊 Total de Colunas: {df_final.shape[1]}")
    print("-" * 50)
    
    # 1. Verifica se a coluna ANO existe e mostra a contagem
    if 'ano' in df_final.columns:
        print("📅 Contagem por ANO:")
        try:
            print(df_final['ano'].value_counts(dropna=False).sort_index())
        except Exception as e:
            print(f"⚠️ Erro ao contar anos: {e}")
    else:
        print("⚠️ Coluna 'ano' não encontrada.")

    print("-" * 50)

    # 2. Mostra a "cara" dos dados
    print("👁️ Amostra (Top 5):")
    display(df_final.head())
    
    # 3. Mostra os tipos de dados
    print("\n📋 Tipos de Dados:")
    print(df_final.dtypes)
else:
    print("Sem dados para exibir. Rode a Célula 2 novamente.")

--------------------------------------------------
📊 Total de Linhas: 73027
📊 Total de Colunas: 16
--------------------------------------------------
📅 Contagem por ANO:
ano
2021    14451
2022    37271
2023    21305
Name: count, dtype: int64
--------------------------------------------------
👁️ Amostra (Top 5):


,idatendimento,ehacompanhante,estadoorigem,faixaetaria,tipohospedagem,tipotransporte,municipointeresse,observacao,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,SIM,DISTRITO FEDERAL,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,SIM,CEARA,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,ACESSO A BOA VIAGEM.,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SIM,SAO PAULO,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0



📋 Tipos de Dados:
idatendimento         object
ehacompanhante        object
estadoorigem          object
faixaetaria           object
tipohospedagem        object
tipotransporte        object
municipointeresse     object
observacao            object
paisorigem            object
sexo                  object
tempoestadia          object
tipoatendimento       object
localatendimento      object
nacionalidade         object
ano                    int64
mes                  float64
dtype: object


In [33]:
# CÉLULA 3.2: Investigação de Nulos

if 'df_final' in locals() and not df_final.empty:
    # 1. Filtra as linhas onde o ANO é nulo, zero ou vazio
    # O comando .isna() pega nulos (NaN)
    filtro_sem_ano = (df_final['ano'].isna()) | (df_final['ano'] == 0) | (df_final['ano'] == '')
    
    df_sem_ano = df_final[filtro_sem_ano]
    
    qtd_sem_ano = len(df_sem_ano)
    print(f"🔎 Total de linhas sem Ano: {qtd_sem_ano}")
    print("-" * 40)

    if qtd_sem_ano > 0:
        # 2. Verifica quantos têm MÊS preenchido vs MÊS nulo
        # .notna() conta quem TEM valor, .isna() conta quem NÃO TEM
        qtd_com_mes = df_sem_ano['mes'].notna().sum()
        qtd_sem_mes = df_sem_ano['mes'].isna().sum()
        
        print(f"Dentro dessas {qtd_sem_ano} linhas:")
        print(f"✅ Têm o Mês preenchido: {qtd_com_mes}")
        print(f"❌ Mês também é nulo:     {qtd_sem_mes}")
        
        print("-" * 40)
        print("Distribuição dos valores encontrados na coluna 'mes':")
        # Mostra quais meses aparecem (ex: 1, 2, 12...) ou se só tem NaN
        print(df_sem_ano['mes'].value_counts(dropna=False))
    else:
        print("Tudo certo! Não há linhas com ano nulo ou zero.")
else:
    print("DataFrame vazio.")

🔎 Total de linhas sem Ano: 0
----------------------------------------
Tudo certo! Não há linhas com ano nulo ou zero.


In [34]:
# CÉLULA 3.3: Verificação de Anos sem Mês

if 'df_final' in locals() and not df_final.empty:
    # 1. Filtra apenas as linhas que TÊM ano válido (diferente de 0 e nulo)
    filtro_com_ano = (df_final['ano'].notna()) & (df_final['ano'] != 0) & (df_final['ano'] != '')
    df_com_ano = df_final[filtro_com_ano]
    
    total_com_ano = len(df_com_ano)
    print(f"✅ Total de linhas com Ano válido: {total_com_ano}")
    print("-" * 40)

    # 2. Dessas linhas com ano, quantas NÃO têm mês?
    # Consideramos 'sem mês' se for NaN, 0 ou vazio
    filtro_sem_mes = (df_com_ano['mes'].isna()) | (df_com_ano['mes'] == 0) | (df_com_ano['mes'] == '')
    
    df_ano_sem_mes = df_com_ano[filtro_sem_mes]
    qtd_problema = len(df_ano_sem_mes)
    
    print(f"⚠️ Linhas com Ano, mas SEM Mês: {qtd_problema}")
    
    if qtd_problema > 0:
        print("-" * 40)
        print("Distribuição do problema por Ano:")
        # Mostra quais anos estão com meses faltando
        print(df_ano_sem_mes['ano'].value_counts().sort_index())
    else:
        print("🎉 Ótima notícia: Todos os registros com ano têm mês preenchido!")
else:
    print("DataFrame vazio.")

✅ Total de linhas com Ano válido: 73027
----------------------------------------
⚠️ Linhas com Ano, mas SEM Mês: 0
🎉 Ótima notícia: Todos os registros com ano têm mês preenchido!


In [35]:
# CÉLULA 3.4: Limpeza de Registros Vazios

if 'df_final' in locals() and not df_final.empty:
    linhas_antes = len(df_final)
    
    # 1. Cria o filtro para MANTER apenas o que tem Ano válido
    # Mantemos linhas onde 'ano' NÃO é nulo E 'ano' é diferente de 0
    # O comando .copy() é usado para evitar avisos de "SettingWithCopy" do Pandas
    df_final = df_final[ (df_final['ano'].notna()) & (df_final['ano'] != 0) ].copy()
    
    linhas_depois = len(df_final)
    linhas_removidas = linhas_antes - linhas_depois
    
    print("-" * 40)
    print(f"📉 Total Inicial:   {linhas_antes}")
    print(f"🗑️ Linhas Removidas: {linhas_removidas}")
    print(f"✅ Total Final:     {linhas_depois}")
    print("-" * 40)
    
    if linhas_removidas == 2096:
        print("Sucesso! Exatamente as 2096 linhas problemáticas foram removidas.")
    else:
        print(f"Atenção: Foram removidas {linhas_removidas} linhas. Verifique se o número bate com o esperado.")

    # Resetar o índice para organizar a tabela
    df_final.reset_index(drop=True, inplace=True)
else:
    print("DataFrame vazio.")

----------------------------------------
📉 Total Inicial:   73027
🗑️ Linhas Removidas: 0
✅ Total Final:     73027
----------------------------------------
Atenção: Foram removidas 0 linhas. Verifique se o número bate com o esperado.


In [37]:
# CÉLULA 3.5: Auditoria de Colunas Vazias e Amostragem

if 'df_final' in locals() and not df_final.empty:
    # 1. Definição das colunas alvo
    colunas_alvo = [
        'qtdadoleslentes', 
        'qtdadultos', 
        'qtdcriancas', 
        'qtdidosos', 
        'cidade', 
        'deslocamento', 
        'informacao', 
        'motivoviagem'
    ]
    
    # Filtra para pegar apenas as que existem de fato no DataFrame
    colunas_existentes = [c for c in colunas_alvo if c in df_final.columns]
    
    print(f"📊 Relatório de Preenchimento ({len(colunas_existentes)} colunas analisadas):")
    print("-" * 80)
    
    # 2. Loop de Análise Estatística
    analise = []
    for col in colunas_existentes:
        total = len(df_final)
        
        # Consideramos "Vazio" se for: NaN, None, string vazia ou 'NAO INFORMADO'
        # Se for numérico, 0 pode ser considerado preenchimento válido, então contamos apenas NaN
        if pd.api.types.is_numeric_dtype(df_final[col]):
            nulos = df_final[col].isna().sum()
            uteis = total - nulos
        else:
            # Para texto, consideramos 'NAO INFORMADO' como vazio/inútil para análise
            nulos = df_final[col].isna().sum() + len(df_final[df_final[col] == 'NAO INFORMADO']) + len(df_final[df_final[col] == ''])
            uteis = total - nulos
            
        perc = (uteis / total) * 100
        
        analise.append({
            'Coluna': col,
            'Total': total,
            'Preenchidos': uteis,
            'Vazios/Nulos': nulos,
            '% Útil': f"{perc:.2f}%"
        })
    
    # Mostra a tabela de porcentagens
    display(pd.DataFrame(analise))
    
    print("\n" + "=" * 80)
    print("👀 PREVIEW DOS DADOS (Primeiras 15 linhas dessas colunas):")
    print("=" * 80)
    
    # 3. Print Integrado (Amostra dos dados)
    # Mostra apenas as colunas que estamos analisando para facilitar a leitura
    display(df_final[colunas_existentes].head(15))
    
    # Dica extra: Verifica se sobrou alguma coluna da lista que não estava no DF
    faltantes = set(colunas_alvo) - set(colunas_existentes)
    if faltantes:
        print(f"\n⚠️ Atenção: As colunas {faltantes} não foram encontradas no DataFrame.")

else:
    print("Sem dados para analisar.")

📊 Relatório de Preenchimento (0 colunas analisadas):
--------------------------------------------------------------------------------


""



👀 PREVIEW DOS DADOS (Primeiras 15 linhas dessas colunas):


""
0
1
2
3
4
5
6
7
8
9



⚠️ Atenção: As colunas {'qtdadoleslentes', 'deslocamento', 'motivoviagem', 'qtdidosos', 'qtdcriancas', 'qtdadultos', 'informacao', 'cidade'} não foram encontradas no DataFrame.


In [38]:
# CÉLULA 3.6: Drop de Colunas Vazias

if 'df_final' in locals() and not df_final.empty:
    # Lista de colunas para remover (baseado na sua análise anterior)
    colunas_para_remover = [
        'qtdadoleslentes', 
        'qtdadultos', 
        'qtdcriancas', 
        'qtdidosos', 
        'cidade', 
        'deslocamento', 
        'informacao', 
        'motivoviagem'
    ]
    
    # Identifica quais dessas colunas realmente existem no DataFrame (segurança)
    colunas_existentes = [c for c in colunas_para_remover if c in df_final.columns]
    
    if colunas_existentes:
        print(f"🗑️ Removendo {len(colunas_existentes)} colunas vazias...")
        
        # O comando drop remove as colunas
        df_final.drop(columns=colunas_existentes, inplace=True)
        
        print("✅ Colunas removidas com sucesso!")
        print("-" * 50)
        print("Colunas Restantes na Tabela:")
        print(df_final.columns.tolist())
    else:
        print("⚠️ As colunas já foram removidas ou não existem.")

    print("-" * 50)
    # Prévia final antes do banco
    print("👁️ Visualização Final (Limpa):")
    display(df_final.head())

else:
    print("Sem dados.")

⚠️ As colunas já foram removidas ou não existem.
--------------------------------------------------
👁️ Visualização Final (Limpa):


,idatendimento,ehacompanhante,estadoorigem,faixaetaria,tipohospedagem,tipotransporte,municipointeresse,observacao,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,SIM,DISTRITO FEDERAL,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,SIM,CEARA,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,ACESSO A BOA VIAGEM.,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SIM,SAO PAULO,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0


In [39]:
# CÉLULA 3.7: Auditoria de 'Acompanhantes', 'Turistas' e 'Município Interesse'

if 'df_final' in locals() and not df_final.empty:
    # Lista de colunas para investigar
    # Nota: Usei os nomes prováveis após a padronização (minúsculos)
    colunas_alvo = ['qtdacompanhantes', 'qtdturistas', 'municipointeresse']
    
    # Filtra apenas as que existem no DataFrame
    colunas_existentes = [c for c in colunas_alvo if c in df_final.columns]
    
    print(f"📊 Analisando {len(colunas_existentes)} colunas...\n")
    
    analise = []
    
    for col in colunas_existentes:
        total = len(df_final)
        
        # Lógica para contar vazios
        if pd.api.types.is_numeric_dtype(df_final[col]):
            # Se for número, conta apenas os NaN (nulos reais)
            nulos = df_final[col].isna().sum()
            uteis = total - nulos
        else:
            # Se for texto, considera também 'NAO INFORMADO' e strings vazias
            nulos = df_final[col].isna().sum() + \
                    len(df_final[df_final[col] == 'NAO INFORMADO']) + \
                    len(df_final[df_final[col] == ''])
            uteis = total - nulos
            
        perc = (uteis / total) * 100
        
        analise.append({
            'Coluna': col,
            'Total Linhas': total,
            'Preenchidos': uteis,
            'Vazios/Nulos': nulos,
            '% Útil': f"{perc:.2f}%"
        })
    
    # Exibe a tabela resumo
    display(pd.DataFrame(analise))
    
    print("\n" + "=" * 80)
    print("👀 PREVIEW (Amostra dos dados):")
    print("=" * 80)
    
    # Mostra uma amostra das colunas para você ver o conteúdo
    display(df_final[colunas_existentes].head(20))
    
    # Alerta se alguma coluna não foi encontrada
    faltantes = set(colunas_alvo) - set(colunas_existentes)
    if faltantes:
        print(f"\n⚠️ Colunas não encontradas: {faltantes}")

else:
    print("Sem dados para analisar.")

📊 Analisando 1 colunas...



,Coluna,Total Linhas,Preenchidos,Vazios/Nulos,% Útil
0,municipointeresse,73027,73027,0,100.00%



👀 PREVIEW (Amostra dos dados):


,municipointeresse
0,NAO PREENCHEU
1,NAO PREENCHEU
2,NAO PREENCHEU
3,NAO PREENCHEU
4,NAO PREENCHEU
5,NAO PREENCHEU
6,NAO PREENCHEU
7,NAO PREENCHEU
8,NAO PREENCHEU
9,NAO PREENCHEU



⚠️ Colunas não encontradas: {'qtdturistas', 'qtdacompanhantes'}


In [40]:
# CÉLULA 3.8: Remoção de 'Acompanhantes' e 'Turistas'

if 'df_final' in locals() and not df_final.empty:
    # Lista exata do que será removido
    colunas_para_remover = ['qtdacompanhantes', 'qtdturistas']
    
    # Verifica quais dessas colunas realmente existem no DataFrame (segurança)
    colunas_existentes = [c for c in colunas_para_remover if c in df_final.columns]
    
    if colunas_existentes:
        print(f"🗑️ Removendo {len(colunas_existentes)} colunas: {colunas_existentes}...")
        
        # Remove as colunas
        df_final.drop(columns=colunas_existentes, inplace=True)
        
        print("✅ Colunas removidas com sucesso!")
    else:
        print("⚠️ As colunas já foram removidas ou não existem no DataFrame.")

    print("-" * 50)
    
    # Mostra as colunas que sobraram para você conferir
    print("📋 Colunas Restantes:")
    print(df_final.columns.tolist())
    
    print("-" * 50)
    # Prévia dos dados limpos
    print("👁️ Visualização Atual:")
    display(df_final.head())

else:
    print("Sem dados.")

⚠️ As colunas já foram removidas ou não existem no DataFrame.
--------------------------------------------------
📋 Colunas Restantes:
['idatendimento', 'ehacompanhante', 'estadoorigem', 'faixaetaria', 'tipohospedagem', 'tipotransporte', 'municipointeresse', 'observacao', 'paisorigem', 'sexo', 'tempoestadia', 'tipoatendimento', 'localatendimento', 'nacionalidade', 'ano', 'mes']
--------------------------------------------------
👁️ Visualização Atual:


,idatendimento,ehacompanhante,estadoorigem,faixaetaria,tipohospedagem,tipotransporte,municipointeresse,observacao,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,SIM,DISTRITO FEDERAL,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,SIM,CEARA,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,ACESSO A BOA VIAGEM.,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SIM,SAO PAULO,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,NAO INFORMADO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0


In [41]:
# CÉLULA 3.9: Auditoria de IDs Duplicados

if 'df_final' in locals() and not df_final.empty:
    # Nome da coluna que identifica cada registro unicamente
    coluna_id = 'idatendimento' 
    
    if coluna_id in df_final.columns:
        # Conta quantos IDs aparecem mais de uma vez
        qtd_duplicados = df_final[coluna_id].duplicated().sum()
        total_registros = len(df_final)
        
        print(f"📊 Análise de Duplicidade na coluna '{coluna_id}':")
        print("-" * 50)
        print(f"Total de Registros: {total_registros}")
        print(f"IDs Únicos:         {df_final[coluna_id].nunique()}")
        print(f"🔴 IDs Duplicados:  {qtd_duplicados}")
        print("-" * 50)
        
        if qtd_duplicados > 0:
            print("👁️ Amostra dos registros duplicados (Top 10):")
            # Mostra as linhas duplicadas (keep=False mostra todas as ocorrências da duplicata)
            # Ordenamos pelo ID para você ver os pares/trios juntos
            duplicados_df = df_final[df_final[coluna_id].duplicated(keep=False)].sort_values(by=coluna_id)
            display(duplicados_df.head(10))
            
            print("\n💡 Dica: Se houver duplicatas, geralmente removemos mantendo apenas o registro mais recente ou o primeiro.")
        else:
            print("✅ Ótimo! Não há IDs duplicados na base.")
            
    else:
        print(f"⚠️ Coluna '{coluna_id}' não encontrada. Verifique se o nome mudou na padronização.")
        print("Colunas disponíveis:", df_final.columns.tolist())
else:
    print("Sem dados para analisar.")

📊 Análise de Duplicidade na coluna 'idatendimento':
--------------------------------------------------
Total de Registros: 73027
IDs Únicos:         73027
🔴 IDs Duplicados:  0
--------------------------------------------------
✅ Ótimo! Não há IDs duplicados na base.


In [42]:
# CÉLULA 3.10: Contagem de Tipos e Locais de Atendimento

if 'df_final' in locals() and not df_final.empty:
    colunas_para_analisar = ['tipoatendimento', 'localatendimento']
    
    for col in colunas_para_analisar:
        # Verifica se a coluna existe (usando o nome padronizado)
        if col in df_final.columns:
            print(f"📊 ANÁLISE DA COLUNA: '{col}'")
            print("-" * 60)
            
            # Conta os valores únicos
            qtd_unicos = df_final[col].nunique()
            print(f"🔢 Quantidade de tipos distintos encontrados: {qtd_unicos}")
            print("-" * 60)
            
            # Cria uma tabela de frequência com Contagem e Porcentagem
            contagem = df_final[col].value_counts(dropna=False)
            porcentagem = df_final[col].value_counts(normalize=True, dropna=False) * 100
            
            # Monta um DataFrame bonitinho para exibir
            tabela_resumo = pd.DataFrame({
                'Total de Registros': contagem,
                'Porcentagem (%)': porcentagem.round(2)
            })
            
            display(tabela_resumo)
            print("\n" + "="*60 + "\n")
            
        else:
            print(f"⚠️ Coluna '{col}' não encontrada no DataFrame.")
else:
    print("Sem dados para analisar.")

📊 ANÁLISE DA COLUNA: 'tipoatendimento'
------------------------------------------------------------
🔢 Quantidade de tipos distintos encontrados: 2
------------------------------------------------------------


,Total de Registros,Porcentagem (%)
tipoatendimento,,
PRESENCIAL,72574,99.38
TELEFONE,453,0.62




📊 ANÁLISE DA COLUNA: 'localatendimento'
------------------------------------------------------------
🔢 Quantidade de tipos distintos encontrados: 16
------------------------------------------------------------


,Total de Registros,Porcentagem (%)
localatendimento,,
CAT AEROPORTO,17205,23.56
CAT PRAÇA DO ARSENAL,16390,22.44
CAT SHOPPING CENTER RECIFE,14305,19.59
CATS TEMPORÁRIOS,7532,10.31
CAT TERMINAL INTEGRADO DE PASSAGEIROS,4492,6.15
CAT PRAÇA DE BOA VIAGEM,3461,4.74
BALCÃO TERMINAL MARÍTIMO,3256,4.46
CAT AMBIENTAL,1715,2.35
CAT CRIATIVO,1370,1.88


In [43]:
# CÉLULA 3.11: Análise Detalhada da Coluna 'observacao'

if 'df_final' in locals() and not df_final.empty:
    col = 'observacao'
    
    # Verifica se a coluna existe (lembrando da padronização para minúsculo)
    if col in df_final.columns:
        print(f"📊 ANÁLISE DA COLUNA: '{col}'")
        print("-" * 60)
        
        total_linhas = len(df_final)
        
        # Identifica o que é considerado "Vazio"
        # Nulos reais, strings vazias ou a tag 'NAO INFORMADO' que aplicamos antes
        filtro_vazio = df_final[col].isna() | \
                       (df_final[col] == '') | \
                       (df_final[col] == 'NAO INFORMADO') | \
                       (df_final[col].astype(str).str.strip() == '')

        qtd_vazios = filtro_vazio.sum()
        qtd_preenchidos = total_linhas - qtd_vazios
        
        print(f"Total de Linhas:            {total_linhas}")
        print(f"📝 Registros com Conteúdo:  {qtd_preenchidos}")
        print(f"⚪ Vazios / Não Informados: {qtd_vazios}")
        
        perc = (qtd_preenchidos / total_linhas) * 100
        print(f"📊 Taxa de Preenchimento:   {perc:.2f}%")
        
        print("-" * 60)
        
        if qtd_preenchidos > 0:
            print("👁️ Amostra de Observações Reais (Top 15):")
            # Mostra apenas as linhas que têm conteúdo real
            amostra = df_final[~filtro_vazio][col].head(15).tolist()
            for i, obs in enumerate(amostra, 1):
                print(f"{i}. {obs}")
    else:
        print(f"⚠️ Coluna '{col}' não encontrada. Verifique se o nome está correto.")
else:
    print("Sem dados para analisar.")

📊 ANÁLISE DA COLUNA: 'observacao'
------------------------------------------------------------
Total de Linhas:            73027
📝 Registros com Conteúdo:  13639
⚪ Vazios / Não Informados: 59388
📊 Taxa de Preenchimento:   18.68%
------------------------------------------------------------
👁️ Amostra de Observações Reais (Top 15):
1. ACESSO A BOA VIAGEM.
2. PASSAPORTE PE
3. PASSAPORTE PE
4. PASSAPORTE PERNAMBUCO
5. PASSAPORTE PERNAMBUCO
6. PASSAPORTE PERNAMBUCO
7. PASSAPORTE PERNAMBUCO
8. PASSAPORTE PERNAMBUCO
9. PASSAPORTE PERNAMBUCO
10. PASSAPORTE PERNAMBUCO
11. PASSAPORTE PERNAMBUCO
12. PASSAPORTE PERNAMBUCO
13. PASSAPORTE PERNAMBUCO
14. PASSAPORTE PERNAMBUCO
15. PASSAPORTES PE (02)


In [44]:
# CÉLULA 3.12: Remoção da coluna 'observacao'

if 'df_final' in locals() and not df_final.empty:
    coluna_alvo = 'observacao'
    
    # Verifica se a coluna existe antes de tentar remover
    if coluna_alvo in df_final.columns:
        print(f"🗑️ Removendo a coluna '{coluna_alvo}'...")
        
        # Remove a coluna
        df_final.drop(columns=[coluna_alvo], inplace=True)
        
        print(f"✅ Coluna '{coluna_alvo}' removida com sucesso!")
        
        print("-" * 50)
        # Mostra as colunas restantes para confirmação final
        print("📋 Colunas Restantes na Tabela:")
        print(df_final.columns.tolist())
    else:
        print(f"⚠️ A coluna '{coluna_alvo}' já não existe no DataFrame.")

    print("-" * 50)
    # Prévia final
    print("👁️ Visualização Atual:")
    display(df_final.head())

else:
    print("Sem dados.")

🗑️ Removendo a coluna 'observacao'...
✅ Coluna 'observacao' removida com sucesso!
--------------------------------------------------
📋 Colunas Restantes na Tabela:
['idatendimento', 'ehacompanhante', 'estadoorigem', 'faixaetaria', 'tipohospedagem', 'tipotransporte', 'municipointeresse', 'paisorigem', 'sexo', 'tempoestadia', 'tipoatendimento', 'localatendimento', 'nacionalidade', 'ano', 'mes']
--------------------------------------------------
👁️ Visualização Atual:


,idatendimento,ehacompanhante,estadoorigem,faixaetaria,tipohospedagem,tipotransporte,municipointeresse,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,SIM,DISTRITO FEDERAL,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,SIM,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,SIM,CEARA,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SIM,SAO PAULO,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0


In [45]:
# CÉLULA 3.13: Análise da coluna 'ehacompanhante'

if 'df_final' in locals() and not df_final.empty:
    coluna_alvo = 'ehacompanhante'
    
    # Verifica se a coluna existe (lembre-se da padronização para minúsculo)
    if coluna_alvo in df_final.columns:
        print(f"📊 ANÁLISE DA COLUNA: '{coluna_alvo}'")
        print("-" * 60)
        
        # Conta valores únicos e nulos
        total = len(df_final)
        nulos = df_final[coluna_alvo].isna().sum()
        preenchidos = total - nulos
        
        print(f"Total de Registros: {total}")
        print(f"Nulos / Vazios:     {nulos}")
        print("-" * 60)
        
        # Tabela de Frequência (Quantos 'Sim', quantos 'Não', etc.)
        contagem = df_final[coluna_alvo].value_counts(dropna=False)
        porcentagem = df_final[coluna_alvo].value_counts(normalize=True, dropna=False) * 100
        
        resumo = pd.DataFrame({
            'Total': contagem,
            'Porcentagem (%)': porcentagem.round(2)
        })
        
        display(resumo)
        
        print("\n" + "="*60)
        print("💡 Dica: Se 99% for 'Sim' ou 'Não', a coluna pode ter baixa variância.")
    else:
        print(f"⚠️ Coluna '{coluna_alvo}' não encontrada. Verifique a grafia.")
else:
    print("Sem dados para analisar.")

📊 ANÁLISE DA COLUNA: 'ehacompanhante'
------------------------------------------------------------
Total de Registros: 73027
Nulos / Vazios:     0
------------------------------------------------------------


,Total,Porcentagem (%)
ehacompanhante,,
SIM,73027,100.0



💡 Dica: Se 99% for 'Sim' ou 'Não', a coluna pode ter baixa variância.


In [46]:
# CÉLULA 3.14: Remoção da coluna 'ehacompanhante'

if 'df_final' in locals() and not df_final.empty:
    coluna_alvo = 'ehacompanhante'
    
    # Verifica se a coluna existe antes de tentar remover
    if coluna_alvo in df_final.columns:
        print(f"🗑️ Removendo a coluna '{coluna_alvo}' (Baixa variância/Redundante)...")
        
        # Remove a coluna
        df_final.drop(columns=[coluna_alvo], inplace=True)
        
        print(f"✅ Coluna '{coluna_alvo}' removida com sucesso!")
        
        print("-" * 50)
        # Mostra as colunas restantes para confirmação final
        print("📋 Colunas Restantes na Tabela:")
        print(df_final.columns.tolist())
    else:
        print(f"⚠️ A coluna '{coluna_alvo}' já não existe no DataFrame.")

    print("-" * 50)
    # Prévia final para garantir que a coluna sumiu
    print("👁️ Visualização Atual:")
    display(df_final.head())

else:
    print("Sem dados.")

🗑️ Removendo a coluna 'ehacompanhante' (Baixa variância/Redundante)...
✅ Coluna 'ehacompanhante' removida com sucesso!
--------------------------------------------------
📋 Colunas Restantes na Tabela:
['idatendimento', 'estadoorigem', 'faixaetaria', 'tipohospedagem', 'tipotransporte', 'municipointeresse', 'paisorigem', 'sexo', 'tempoestadia', 'tipoatendimento', 'localatendimento', 'nacionalidade', 'ano', 'mes']
--------------------------------------------------
👁️ Visualização Atual:


,idatendimento,estadoorigem,faixaetaria,tipohospedagem,tipotransporte,municipointeresse,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,DISTRITO FEDERAL,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,DISTRITO FEDERAL,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,CEARA,ADULTO,NAO PREENCHEU,AVIAO,NAO PREENCHEU,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SAO PAULO,ADULTO,NAO-INFORMOU,AVIAO,NAO PREENCHEU,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0


In [48]:
# CÉLULA 3.15: Análise de Perfil (Hospedagem, Transporte e Faixa Etária)

if 'df_final' in locals() and not df_final.empty:
    # Lista de colunas para analisar
    colunas_perfil = ['tipohospedagem', 'tipotransporte', 'faixaetaria']
    
    # Filtra apenas as que existem (segurança)
    colunas_existentes = [c for c in colunas_perfil if c in df_final.columns]
    
    print(f"📊 Análise de Perfil do Turista ({len(colunas_existentes)} colunas)")
    print("=" * 70)
    
    for col in colunas_existentes:
        print(f"\n🔎 COLUNA: {col.upper()}")
        print("-" * 30)
        
        # 1. Contagem de Nulos e Preenchimento
        total = len(df_final)
        # Considera vazio: NaN, None, string vazia ou 'NAO INFORMADO'
        nulos = df_final[col].isna().sum() + \
                len(df_final[df_final[col] == 'NAO INFORMADO']) + \
                len(df_final[df_final[col] == ''])
                
        preenchidos = total - nulos
        perc_util = (preenchidos / total) * 100
        
        print(f"Total: {total} | Preenchidos: {preenchidos} ({perc_util:.1f}%) | Vazios: {nulos}")
        
        # 2. Distribuição dos Valores (Top 10)
        print("\n🏆 Top 10 Valores Mais Comuns:")
        contagem = df_final[col].value_counts(normalize=True).head(10) * 100
        
        # Mostra formatado
        for item, perc in contagem.items():
            print(f"   - {item}: {perc:.1f}%")
            
        # --- DETALHAMENTO ESPECÍFICO PARA TRANSPORTE ---
        if col == 'tipotransporte':
            print("\n   🚌 DETALHAMENTO EXTRA (TRANSPORTE):")
            print("   " + "-"*40)
            
            # A. Lista de todos os valores únicos (para caçar erros de digitação)
            unicos = df_final[col].dropna().unique()
            print(f"   Valores únicos encontrados ({len(unicos)}):")
            print(f"   {sorted([str(u) for u in unicos])}")
            
            # B. Comparativo Temporal (Transporte x Ano)
            if 'ano' in df_final.columns:
                print("\n   📅 Evolução do Transporte por Ano (Tabela Cruzada):")
                try:
                    # Cria uma tabela dinâmica simples
                    tabela_ano = pd.crosstab(df_final[col], df_final['ano'])
                    display(tabela_ano)
                except Exception as e:
                    print(f"   Erro ao gerar tabela por ano: {e}")

        print("." * 70)

else:
    print("Sem dados para analisar.")

📊 Análise de Perfil do Turista (3 colunas)

🔎 COLUNA: TIPOHOSPEDAGEM
------------------------------
Total: 73027 | Preenchidos: 73027 (100.0%) | Vazios: 0

🏆 Top 10 Valores Mais Comuns:
   - NAO PREENCHEU: 71.0%
   - HOTEL: 18.6%
   - AMIGOS-PARENTES: 3.6%
   - NAO-INFORMOU: 3.0%
   - POUSADA: 1.7%
   - OUTROS: 1.3%
   - FLAT: 0.3%
   - CASA-ALUGADA: 0.3%
   - ALBERGUE: 0.2%
......................................................................

🔎 COLUNA: TIPOTRANSPORTE
------------------------------
Total: 73027 | Preenchidos: 73027 (100.0%) | Vazios: 0

🏆 Top 10 Valores Mais Comuns:
   - NAO PREENCHEU: 63.8%
   - AVIAO: 27.1%
   - ONIBUS: 3.7%
   - NAVIO: 2.1%
   - NAO-INFORMOU: 1.6%
   - CARRO: 1.2%
   - OUTROS: 0.3%

   🚌 DETALHAMENTO EXTRA (TRANSPORTE):
   ----------------------------------------
   Valores únicos encontrados (7):
   ['AVIAO', 'CARRO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAVIO', 'ONIBUS', 'OUTROS']

   📅 Evolução do Transporte por Ano (Tabela Cruzada):


ano,2021,2022,2023
tipotransporte,,,
AVIAO,3238,8262,8310
CARRO,127,396,384
NAO PREENCHEU,10694,26848,9075
NAO-INFORMOU,215,326,650
NAVIO,2,63,1496
ONIBUS,132,1336,1231
OUTROS,43,40,159


......................................................................

🔎 COLUNA: FAIXAETARIA
------------------------------
Total: 73027 | Preenchidos: 73027 (100.0%) | Vazios: 0

🏆 Top 10 Valores Mais Comuns:
   - ADULTO: 99.8%
   - CRIANCA: 0.1%
   - IDOSO: 0.1%
   - ADOLESCENTE: 0.0%
......................................................................


In [49]:
# CÉLULA 3.16: Remoção de Faixa Etária e Padronização Global

if 'df_final' in locals() and not df_final.empty:
    # 1. Remover coluna 'faixaetaria' (Baixa variância: 99% Adulto)
    coluna_alvo = 'faixaetaria'
    if coluna_alvo in df_final.columns:
        df_final.drop(columns=[coluna_alvo], inplace=True)
        print(f"🗑️ Coluna '{coluna_alvo}' removida com sucesso.")
    else:
        print(f"⚠️ Coluna '{coluna_alvo}' já não existe.")

    print("-" * 50)

    # 2. Padronização Global de 'Não Informado' -> 'DESCONHECIDO'
    # Lista de variações encontradas nos dados (baseado no CSV e limpezas anteriores)
    termos_para_substituir = [
        'NAO INFORMADO', 
        'NAO PREENCHEU', 
        'NAO-INFORMOU',
        'NAO_INFORMADO',
        'SEM INFORMAÇÃO',
        'SEM INFORMACAO'
    ]
    
    print(f"🔄 Substituindo variações de nulos por 'DESCONHECIDO'...")
    
    # Seleciona apenas colunas de texto para evitar erro em colunas numéricas
    cols_texto = df_final.select_dtypes(include=['object']).columns
    
    # Aplica a substituição em massa
    for col in cols_texto:
        # Substitui a lista de termos pelo padrão único
        df_final[col] = df_final[col].replace(termos_para_substituir, 'DESCONHECIDO')
        
    print("✅ Padronização concluída!")
    
    print("-" * 50)
    # Prévia para conferir se 'DESCONHECIDO' aparece
    print("👁️ Visualização Atual (Amostra):")
    display(df_final.head(10))

else:
    print("Sem dados.")

🗑️ Coluna 'faixaetaria' removida com sucesso.
--------------------------------------------------
🔄 Substituindo variações de nulos por 'DESCONHECIDO'...
✅ Padronização concluída!
--------------------------------------------------
👁️ Visualização Atual (Amostra):


,idatendimento,estadoorigem,tipohospedagem,tipotransporte,municipointeresse,paisorigem,sexo,tempoestadia,tipoatendimento,localatendimento,nacionalidade,ano,mes
0,138350,DISTRITO FEDERAL,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
1,138351,DISTRITO FEDERAL,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
2,138352,DISTRITO FEDERAL,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
3,138353,CEARA,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
4,138354,SAO PAULO,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
5,138355,ALAGOAS,AMIGOS-PARENTES,AVIAO,DESCONHECIDO,BRASIL,FEM,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
6,138356,SAO PAULO,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,1 SEMANA,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
7,138357,DISTRITO FEDERAL,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,MASC,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
8,138358,DISTRITO FEDERAL,DESCONHECIDO,AVIAO,DESCONHECIDO,BRASIL,FEM,NAO INFORMOU,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0
9,138359,SAO PAULO,OUTROS,AVIAO,DESCONHECIDO,BRASIL,FEM,1 A 2 DIAS,PRESENCIAL,CAT AEROPORTO,NACIONAL,2021,1.0


In [51]:
# CÉLULA 3.17: Análise Detalhada de 'tempoestadia'

if 'df_final' in locals() and not df_final.empty:
    coluna_alvo = 'tempoestadia'
    
    # Verifica se a coluna existe (lembrando da padronização para minúsculo)
    if coluna_alvo in df_final.columns:
        print(f"📊 ANÁLISE PROFUNDA: '{coluna_alvo}'")
        print("-" * 60)
        
        # 1. Valores Únicos (para ver a bagunça)
        unicos = df_final[coluna_alvo].unique()
        print(f"🔢 Quantidade de valores únicos: {len(unicos)}")
        print("\n📝 Lista de todos os valores encontrados:")
        # Ordena convertendo para string para não dar erro se tiver mistura de numero/texto
        print(sorted([str(u) for u in unicos]))
        
        print("-" * 60)
        
        # 2. Frequência (O que é mais comum?)
        print("🏆 Top 20 Valores Mais Comuns:")
        contagem = df_final[coluna_alvo].value_counts(dropna=False).head(20)
        display(contagem)
        
    else:
        print(f"⚠️ Coluna '{coluna_alvo}' não encontrada.")
else:
    print("Sem dados.")

📊 ANÁLISE PROFUNDA: 'tempoestadia'
------------------------------------------------------------
🔢 Quantidade de valores únicos: 6

📝 Lista de todos os valores encontrados:
['1 A 2 DIAS', '1 SEMANA', '3 A 4 DIAS', '5 A 6 DIAS', 'NAO INFORMOU', 'OUTROS']
------------------------------------------------------------
🏆 Top 20 Valores Mais Comuns:


tempoestadia
NAO INFORMOU    51737
5 A 6 DIAS       4709
3 A 4 DIAS       4510
OUTROS           4445
1 A 2 DIAS       3821
1 SEMANA         3805
Name: count, dtype: int64

In [52]:
# CÉLULA 3.17: Padronização Global e Normalização de 'tempoestadia'

import re

if 'df_final' in locals() and not df_final.empty:
    print("🔄 Iniciando Padronização Global e Normalização de Tempo...")
    print("-" * 60)

    # --- 1. SUBSTUIÇÃO GLOBAL (Reforço) ---
    # Lista completa de termos para transformar em 'DESCONHECIDO'
    termos_nulos = [
        'NAO INFORMADO', 'NAO PREENCHEU', 'NAO-INFORMOU', 'NAO_INFORMADO',
        'SEM INFORMAÇÃO', 'SEM INFORMACAO', 'NAN', 'NULL', 'nan', '', ' '
    ]
    
    # Aplica em TODAS as colunas de texto da tabela
    cols_texto = df_final.select_dtypes(include=['object']).columns
    for col in cols_texto:
        # Primeiro limpa espaços
        df_final[col] = df_final[col].str.strip().str.upper()
        # Substitui os termos pela tag padrão
        df_final[col] = df_final[col].replace(termos_nulos, 'DESCONHECIDO')
        
    print("✅ Substituição global por 'DESCONHECIDO' aplicada em toda a tabela.")

    # --- 2. NORMALIZAÇÃO: TEMPO MÁXIMO DE ESTADIA ---
    col_origem = 'tempoestadia'
    col_destino = 'tempo_maximo_estadia'
    
    if col_origem in df_final.columns:
        # Função inteligente para extrair o máximo em dias
        def extrair_maximo_dias(texto):
            texto = str(texto).upper().strip()
            
            if texto == 'DESCONHECIDO' or texto == 'OUTROS':
                return 'DESCONHECIDO'
            
            # Fatores de conversão (aproximados)
            multiplicador = 1
            if 'SEMANA' in texto:
                multiplicador = 7
            elif 'MES' in texto or 'MÊS' in texto:
                multiplicador = 30
            elif 'ANO' in texto:
                multiplicador = 365
            
            # Encontra todos os números na string (ex: "5 a 6" -> ['5', '6'])
            numeros = re.findall(r'\d+', texto)
            
            if numeros:
                # Converte para inteiros
                numeros_int = [int(n) for n in numeros]
                # Pega o MAIOR número e aplica o multiplicador (ex: 1 semana -> 1*7 = 7)
                maximo = max(numeros_int) * multiplicador
                return int(maximo)
            else:
                return 'DESCONHECIDO'

        # Aplica a função
        df_final[col_destino] = df_final[col_origem].apply(extrair_maximo_dias)
        
        # Remove a coluna antiga (opcional, mas recomendado para limpar)
        # df_final.drop(columns=[col_origem], inplace=True) 
        
        print(f"✅ Coluna '{col_destino}' criada com sucesso.")
        print("-" * 60)
        
        # --- PRÉVIA DO RESULTADO ---
        print("🔎 Comparativo: Original vs Normalizado (Amostra variada):")
        cols_preview = [col_origem, col_destino]
        # Pega uma amostra que tenha valores diferentes de DESCONHECIDO para validar
        amostra = df_final[df_final[col_destino] != 'DESCONHECIDO'][cols_preview].drop_duplicates().head(10)
        display(amostra)
        
        print("\n🔎 Distribuição dos Valores Normalizados:")
        print(df_final[col_destino].value_counts().head(10))
        
    else:
        print(f"⚠️ Coluna '{col_origem}' não encontrada.")

else:
    print("Sem dados.")

🔄 Iniciando Padronização Global e Normalização de Tempo...
------------------------------------------------------------
✅ Substituição global por 'DESCONHECIDO' aplicada em toda a tabela.
✅ Coluna 'tempo_maximo_estadia' criada com sucesso.
------------------------------------------------------------
🔎 Comparativo: Original vs Normalizado (Amostra variada):


,tempoestadia,tempo_maximo_estadia
1,1 SEMANA,7
3,1 A 2 DIAS,2
21,3 A 4 DIAS,4
84,5 A 6 DIAS,6



🔎 Distribuição dos Valores Normalizados:
tempo_maximo_estadia
DESCONHECIDO    56182
6                4709
4                4510
2                3821
7                3805
Name: count, dtype: int64


In [54]:
# CÉLULA 4.0: Modelagem Estratégica (Star Schema)

if 'df_final' in locals() and not df_final.empty:
    print("🌟 ETAPA 4: Gerando Modelo Estrela (Fato e Dimensões)...")
    print("-" * 60)
    
    # --- 1. DIMENSÃO CALENDÁRIO (dim_calendario) ---
    # Tabela com as datas únicas para filtro temporal
    dim_calendario = df_final[['ano', 'mes']].drop_duplicates().reset_index(drop=True)
    dim_calendario['id_tempo'] = dim_calendario.index + 1
    
    # Adiciona o ID na tabela principal para ligar depois
    df_final = df_final.merge(dim_calendario, on=['ano', 'mes'], how='left')
    print("   -> Dimensão 'dim_calendario' criada.")
    
    # --- 2. DIMENSÃO LOCALIZAÇÃO (dim_localizacao) ---
    # Informações sobre a origem do turista
    cols_local = ['paisorigem', 'estadoorigem', 'nacionalidade']
    cols_existentes_local = [c for c in cols_local if c in df_final.columns]
    
    dim_localizacao = df_final[cols_existentes_local].drop_duplicates().reset_index(drop=True)
    dim_localizacao['id_localizacao'] = dim_localizacao.index + 1
    
    df_final = df_final.merge(dim_localizacao, on=cols_existentes_local, how='left')
    print("   -> Dimensão 'dim_localizacao' criada.")

    # --- 3. DIMENSÃO VIAGEM (dim_viagem) ---
    # Características da estadia e logística (incluindo a nova coluna normalizada)
    cols_viagem = ['tipohospedagem', 'tipotransporte', 'tempo_maximo_estadia', 'municipointeresse']
    cols_existentes_viagem = [c for c in cols_viagem if c in df_final.columns]
    
    dim_viagem = df_final[cols_existentes_viagem].drop_duplicates().reset_index(drop=True)
    dim_viagem['id_viagem'] = dim_viagem.index + 1
    
    df_final = df_final.merge(dim_viagem, on=cols_existentes_viagem, how='left')
    print("   -> Dimensão 'dim_viagem' criada.")

    # --- 4. DIMENSÃO ATENDIMENTO (dim_atendimento) ---
    # Dados sobre o posto de atendimento
    cols_atendimento = ['localatendimento', 'tipoatendimento']
    cols_existentes_atend = [c for c in cols_atendimento if c in df_final.columns]
    
    dim_atendimento_info = df_final[cols_existentes_atend].drop_duplicates().reset_index(drop=True)
    dim_atendimento_info['id_atendimento_info'] = dim_atendimento_info.index + 1
    
    df_final = df_final.merge(dim_atendimento_info, on=cols_existentes_atend, how='left')
    print("   -> Dimensão 'dim_atendimento' criada.")

    # --- 5. TABELA FATO (fato_atendimentos) ---
    # Contém apenas as chaves (IDs) e métricas. É o centro da estrela.
    colunas_fato = [
        'idatendimento',      # Chave primária original (Degenerate Dimension)
        'id_tempo',           # FK para Calendário
        'id_localizacao',     # FK para Localização
        'id_viagem',          # FK para Viagem
        'id_atendimento_info',# FK para Atendimento
        'sexo'                # Perfil (pode ser usado para contagem demográfica)
    ]
    
    # Filtra para garantir que só pegamos colunas que existem
    cols_finais_fato = [c for c in colunas_fato if c in df_final.columns]
    
    fato_atendimentos = df_final[cols_finais_fato].copy()
    
    print("   -> Tabela Fato 'fato_atendimentos' criada.")
    print("-" * 60)
    print("✅ Modelagem concluída! As tabelas estão prontas na memória para carga.")

else:
    print("❌ Sem dados para modelar. Rode as etapas anteriores.")

🌟 ETAPA 4: Gerando Modelo Estrela (Fato e Dimensões)...
------------------------------------------------------------
   -> Dimensão 'dim_calendario' criada.
   -> Dimensão 'dim_localizacao' criada.
   -> Dimensão 'dim_viagem' criada.
   -> Dimensão 'dim_atendimento' criada.
   -> Tabela Fato 'fato_atendimentos' criada.
------------------------------------------------------------
✅ Modelagem concluída! As tabelas estão prontas na memória para carga.


In [55]:
# CÉLULA 4.1: Prévia das Tabelas (Fato e Dimensões)

if 'fato_atendimentos' in locals():
    print("🌟 ESQUEMA ESTRELA GERADO (PRÉVIA) 🌟")
    print("=" * 60)
    
    # 1. Tabela Fato
    print(f"1️⃣ TABELA FATO: fato_atendimentos ({len(fato_atendimentos)} registros)")
    print("   Colunas: chaves estrangeiras (IDs) e métricas")
    display(fato_atendimentos.head())
    print("." * 60)
    
    # 2. Dimensão Calendário
    if 'dim_calendario' in locals():
        print(f"2️⃣ DIMENSÃO: dim_calendario ({len(dim_calendario)} registros)")
        display(dim_calendario.head())
        print("." * 60)
    
    # 3. Dimensão Localização
    if 'dim_localizacao' in locals():
        print(f"3️⃣ DIMENSÃO: dim_localizacao ({len(dim_localizacao)} registros)")
        display(dim_localizacao.head())
        print("." * 60)
    
    # 4. Dimensão Viagem
    if 'dim_viagem' in locals():
        print(f"4️⃣ DIMENSÃO: dim_viagem ({len(dim_viagem)} registros)")
        print("   Nota: Verifique se a coluna 'tempo_maximo_estadia' está aqui.")
        display(dim_viagem.head())
        print("." * 60)

    # 5. Dimensão Atendimento
    if 'dim_atendimento_info' in locals():
        print(f"5️⃣ DIMENSÃO: dim_atendimento ({len(dim_atendimento_info)} registros)")
        display(dim_atendimento_info.head())
    
else:
    print("❌ As tabelas não foram encontradas. Rode o Bloco 4.0 primeiro.")

🌟 ESQUEMA ESTRELA GERADO (PRÉVIA) 🌟
1️⃣ TABELA FATO: fato_atendimentos (73027 registros)
   Colunas: chaves estrangeiras (IDs) e métricas


,idatendimento,id_tempo,id_localizacao,id_viagem,id_atendimento_info,sexo
0,138350,1,1,1,1,MASC
1,138351,1,1,2,1,MASC
2,138352,1,1,1,1,MASC
3,138353,1,2,3,1,FEM
4,138354,1,3,1,1,MASC


............................................................
2️⃣ DIMENSÃO: dim_calendario (36 registros)


,ano,mes,id_tempo
0,2021,1.0,1
1,2021,2.0,2
2,2021,3.0,3
3,2021,4.0,4
4,2021,5.0,5


............................................................
3️⃣ DIMENSÃO: dim_localizacao (145 registros)


,paisorigem,estadoorigem,nacionalidade,id_localizacao
0,BRASIL,DISTRITO FEDERAL,NACIONAL,1
1,BRASIL,CEARA,NACIONAL,2
2,BRASIL,SAO PAULO,NACIONAL,3
3,BRASIL,ALAGOAS,NACIONAL,4
4,BRASIL,PERNAMBUCO,NACIONAL,5


............................................................
4️⃣ DIMENSÃO: dim_viagem (911 registros)
   Nota: Verifique se a coluna 'tempo_maximo_estadia' está aqui.


,tipohospedagem,tipotransporte,tempo_maximo_estadia,municipointeresse,id_viagem
0,DESCONHECIDO,AVIAO,DESCONHECIDO,DESCONHECIDO,1
1,DESCONHECIDO,AVIAO,7,DESCONHECIDO,2
2,DESCONHECIDO,AVIAO,2,DESCONHECIDO,3
3,AMIGOS-PARENTES,AVIAO,DESCONHECIDO,DESCONHECIDO,4
4,OUTROS,AVIAO,2,DESCONHECIDO,5


............................................................
5️⃣ DIMENSÃO: dim_atendimento (24 registros)


,localatendimento,tipoatendimento,id_atendimento_info
0,CAT AEROPORTO,PRESENCIAL,1
1,CAT PRAÇA DO ARSENAL,PRESENCIAL,2
2,CAT TERMINAL INTEGRADO DE PASSAGEIROS,PRESENCIAL,3
3,CAT SHOPPING CENTER RECIFE,PRESENCIAL,4
4,CAT TERMINAL INTEGRADO DE PASSAGEIROS,TELEFONE,5


In [57]:
# CÉLULA 5.0: Carga no PostgreSQL (Load)

from sqlalchemy import create_engine
from urllib.parse import quote_plus  # Import necessário para tratar o '@' na senha

# Verifica se as tabelas existem na memória antes de tentar carregar
if 'fato_atendimentos' in locals() and 'dim_calendario' in locals():
    print("🐘 ETAPA 5: Carga no Banco de Dados 'turismo_recife'...")
    print("-" * 60)
    
    # --- CONFIGURAÇÃO DO BANCO DE DADOS ---
    DB_USER = 'postgres'
    DB_PASS = 'P@celso4364'  # Sua senha com caractere especial
    DB_HOST = 'localhost'
    DB_PORT = '5432'
    DB_NAME = 'turismo_recife'
    
    # Tratamento da senha: converte '@' para '%40' para não quebrar a URL de conexão
    encoded_pass = quote_plus(DB_PASS)
    
    # Cria a string de conexão usando a senha codificada
    string_conexao = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    
    try:
        # Cria a engine de conexão
        engine = create_engine(string_conexao)
        
        # Dicionário mapeando: Nome da Tabela no Banco -> DataFrame na Memória
        tabelas_para_carga = {
            'dim_calendario': dim_calendario,
            'dim_localizacao': dim_localizacao,
            'dim_viagem': dim_viagem,
            'dim_atendimento': dim_atendimento_info,
            'fato_atendimentos': fato_atendimentos
        }
        
        # Loop para carregar cada tabela
        for nome_tabela, df_tabela in tabelas_para_carga.items():
            print(f"   📤 Carregando tabela '{nome_tabela}'...")
            print(f"      -> {len(df_tabela)} registros a serem inseridos.")
            
            # Envia para o SQL
            # if_exists='replace': Cria a tabela se não existir, ou substitui se já existir
            # index=False: Não envia o índice do Pandas como coluna
            df_tabela.to_sql(nome_tabela, engine, if_exists='replace', index=False, chunksize=5000)
            
            print(f"      ✅ Sucesso!")
            
        print("-" * 60)
        print("\n🎉 PROCESSO ETL CONCLUÍDO COM SUCESSO!")
        print("   Todas as tabelas do modelo estrela foram criadas no PostgreSQL.")
        
    except Exception as e:
        print(f"\n❌ Erro crítico na conexão ou carga: {e}")
        print("Dica: Verifique se o banco 'turismo_recife' foi criado e se a senha está correta.")

else:
    print("❌ Erro: As tabelas não foram encontradas na memória. Rode o Bloco 4.0 primeiro.")

🐘 ETAPA 5: Carga no Banco de Dados 'turismo_recife'...
------------------------------------------------------------
   📤 Carregando tabela 'dim_calendario'...
      -> 36 registros a serem inseridos.
      ✅ Sucesso!
   📤 Carregando tabela 'dim_localizacao'...
      -> 145 registros a serem inseridos.
      ✅ Sucesso!
   📤 Carregando tabela 'dim_viagem'...
      -> 911 registros a serem inseridos.
      ✅ Sucesso!
   📤 Carregando tabela 'dim_atendimento'...
      -> 24 registros a serem inseridos.
      ✅ Sucesso!
   📤 Carregando tabela 'fato_atendimentos'...
      -> 73027 registros a serem inseridos.
      ✅ Sucesso!
------------------------------------------------------------

🎉 PROCESSO ETL CONCLUÍDO COM SUCESSO!
   Todas as tabelas do modelo estrela foram criadas no PostgreSQL.
